<a href="https://colab.research.google.com/github/alicelindel3/ibm5100/blob/main/1129/week7_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework7
時系列データを含むデータベースファイル (data.sqlite3) に含まれるデータについて、goods_genre_id ごとに price の平均値を求めてください。時系列データには goods_genre_id の情報は含まれていないので、複数のテーブルを組み合わせて処理する必要があります。  
データの詳細は以下を参照してください。

### データ一覧
今回与えられるデータは data.sqlite3 という sqlite3 形式のデータです。
以下の 2 つのテーブルが含まれています。
* data ... 時系列データ
* goods ... 商品に関する補足情報

### カラム情報一覧
データに含まれるカラムの情報は以下です。
* index ... (店舗, 商品) の各組み合わせに対して割り振られるID
* store_id ... 店舗ID
* goods_id ... 商品ID
* goods_genre_id ... 商品カテゴリID
* units_sold_day ... 商品の一日の売り上げ個数
* units_sold_month ... 商品のひと月の売り上げ個数
* price ... 商品の現在の値段
* yy-mm-dd ... 日付 (yy/mm/dd形式)
* num_month ... 便宜上各月にわりふられた値 (平成25年1月=1, 平成25年2月=2,...,平成27年9月=33)
* goods_name ... 商品名
* store_name ... 店舗名
* goods_genre_name ... 商品カテゴリ名

### 注意事項
- データ処理はSQLiteでの操作を推奨します。補足資料「SQLite_manual.pdf」「SQLite_manual.ipynb」を参照ください。
- SQLiteのインストール/ダウンロードが上手くいかない場合、python上での操作も可能です。補足資料「csv_export_python.pdf」「csv_export_python.ipynb」を参照ください。
- 一部の処理に時間がかかる場合があります。

### 提出/採点
回答は**Omnicampus**上で提出してください。提出するデータの形式は以下のような**csvファイル**となります。
 
goods_genre_id|avg_price
---|---
1|25000
3|26500
4|25000
…|…
196|20000
200|27000


提出ファイルがこの形式に準じていなかった場合にはスコアが1000となるので、その際はファイル形式を確認しましょう。  

In [ ]:
import sqlite3
import csv

In [ ]:
# Google Colaboratoryで作業する場合に実行してください．  
# データの読み込み方が不明な場合は，別途配布している「GoogleColabにおけるドライブのマウントの仕方」と「Google Drive上でのフォルダ構造の説明」のpdfファイルをご覧ください．
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# データベースファイルのパスを渡してください
dbpath = '/content/drive/My Drive/Colab Notebooks/data.sqlite3'

In [ ]:
conn = sqlite3.connect(dbpath)

In [ ]:
cur = conn.cursor()

In [ ]:
# テーブル名の確認
cur.execute('select name from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('data',)
('goods',)


In [ ]:
# カラム名(CREATE TABLE文)の確認
cur.execute('select sql from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)',)
('CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)',)


In [ ]:
# 全情報の確認 (type, name, tbl_name, rootpage, sql)
cur.execute('select * from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('table', 'data', 'data', 2, 'CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)')
('table', 'goods', 'goods', 12, 'CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)')


In [ ]:
cur.execute('select * from data')
print(cur.fetchone()) #1レコード目の取得
print(cur.fetchone()) #2レコード目の取得

('25-01-29', '-33', '43', '17718', '44943', '2')
('25-01-08', '-33', '32', '8648', '14949', '1')


In [ ]:
cur.execute('select * from data')
for row in cur.fetchall():
  print(row)

('25-01-29', '-33', '43', '17718', '44943', '2')
('25-01-08', '-33', '32', '8648', '14949', '1')
('25-01-16', '-33', '53', '10168', '30158', '1')
('25-01-23', '-33', '31', '1005', '9922', '1')
('25-01-12', '-33', '27', '19861', '14988', '0')
('25-01-11', '-33', '26', '18473', '19768', '1')
('25-01-09', '-33', '51', '6096', '170492', '1')
('25-01-04', '-33', '7', '13902', '29793', '1')
('25-01-10', '-33', '23', '17213', '15099', '1')
('25-01-20', '-33', '47', '15105', '129142', '1')
('25-01-18', '-33', '1', '4367', '25964', '2')
('25-01-03', '-33', '43', '2309', '79932', '2')
('25-01-04', '-33', '38', '17405', '14757', '1')
('25-01-25', '-33', '7', '3607', '29863', '1')
('25-01-25', '-33', '27', '16220', '39897', '1')
('25-01-21', '-33', '55', '21760', '49058', '1')
('25-01-25', '-33', '31', '7442', '9749', '1')
('25-01-24', '-33', '48', '13737', '59698', '1')
('25-01-22', '-33', '3', '8096', '41954', '1')
('25-01-26', '-33', '55', '7172', '172289', '1')
('25-01-18', '-33', '23', '465',

In [ ]:
cur.execute('select goods_genre_id , goods_id from goods order by goods_genre_id')
for row in cur.fetchall():
  print(row)

ストリーミング出力は最後の 5000 行に切り捨てられました。
('60', '5439')
('60', '5484')
('60', '5497')
('60', '5499')
('60', '5500')
('60', '5548')
('60', '5554')
('60', '5949')
('60', '5959')
('60', '5965')
('60', '5991')
('60', '6010')
('60', '6011')
('60', '6013')
('60', '6014')
('60', '6017')
('60', '6031')
('60', '6038')
('60', '6044')
('60', '6189')
('60', '6212')
('60', '6215')
('60', '6220')
('60', '6221')
('60', '6236')
('60', '6240')
('60', '6251')
('60', '6259')
('60', '6269')
('60', '6270')
('60', '6294')
('60', '6303')
('60', '6321')
('60', '6322')
('60', '6335')
('60', '6340')
('60', '6349')
('60', '6355')
('60', '6803')
('60', '6826')
('60', '6853')
('60', '6863')
('60', '6869')
('60', '6879')
('60', '6885')
('60', '6888')
('60', '7163')
('60', '7371')
('60', '7384')
('60', '7416')
('60', '7448')
('60', '7450')
('60', '7451')
('60', '7515')
('60', '7568')
('60', '7677')
('60', '7678')
('60', '7992')
('60', '8037')
('60', '8067')
('60', '8166')
('60', '8244')
('60', '8368')
('60', '8601')
('60', '

In [ ]:
cur.execute('select * from data a join goods b on a.goods_id=b.goods_id')
for row in cur.fetchall():
  print(row)

('25-01-29', '-33', '43', '17718', '44943', '2', 'Receiving cash for 1C-line', '17718', '80')
('25-01-08', '-33', '32', '8648', '14949', '1', 'Barbie as the Princess and pop stars (region)', '8648', '41')
('25-01-16', '-33', '53', '10168', '30158', '1', 'Vysotsky Vladimir theatrical sketch on Tagansky 3DVD topics', '10168', '60')
('25-01-23', '-33', '31', '1005', '9922', '1', '3D Action Puzzle "Technique" racing machine', '1005', '68')
('25-01-12', '-33', '27', '19861', '14988', '0', 'Three from Buttermilk (Region)', '19861', '41')
('25-01-11', '-33', '26', '18473', '19768', '1', 'Sat. Dance Paradise 29', '18473', '56')
('25-01-09', '-33', '51', '6096', '170492', '1', 'Rayman Origins [PS Vita, Russian documentation]', '6096', '23')
('25-01-04', '-33', '7', '13902', '29793', '1', 'Leps Grigory Full Throttle (firms).', '13902', '56')
('25-01-10', '-33', '23', '17213', '15099', '1', 'MIDNIGHT IN PARIS', '17213', '41')
('25-01-20', '-33', '47', '15105', '129142', '1', 'microrobot Spider', 

In [ ]:
cur.execute('select goods_genre_id, avg(price) as avg_price from data a join goods b on a.goods_id=b.goods_id group by goods_genre_id ')
for row in cur.fetchall():
  print(row)

('12', 1381663.0)
('15', 1197012.5)
('16', 1315036.0)
('19', 29577.0)
('20', 154165.75)
('21', 248176.0)
('22', 52100.066666666666)
('23', 167440.53846153847)
('24', 164675.7037037037)
('26', 71894.57142857143)
('29', 63498.39473684211)
('3', 168482.4)
('30', 129826.33333333333)
('31', 42573.60625)
('32', 41988.6)
('34', 120571.0)
('36', 222425.0)
('38', 42881.25)
('39', 108544.72222222222)
('4', 47476.5)
('41', 27312.9203187251)
('42', 96667.75)
('44', 28968.470588235294)
('46', 20584.5)
('5', 129025.0)
('50', 34330.857142857145)
('56', 26939.964285714286)
('58', 26590.416666666668)
('59', 92398.5)
('6', 39211.333333333336)
('60', 45314.42857142857)
('61', 171138.33333333334)
('62', 79763.0)
('63', 67526.75)
('64', 44575.181818181816)
('65', 99287.0625)
('66', 30284.083333333332)
('67', 13455.6)
('68', 47454.0)
('7', 158876.2)
('70', 81622.5)
('71', 29752.75)
('73', 142723.16666666666)
('74', 95758.0)
('76', 162571.72222222222)
('77', 149070.0)
('78', 24784.666666666668)
('80', 44943.

In [ ]:
#テーブルから特定の列を取得
cur.execute('select goods_genre_id, avg(price) as avg_price from data a join goods b on a.goods_id=b.goods_id group by goods_genre_id')
with open('out.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [ ]:
# 処理を確定
conn.commit()

In [ ]:
cur.close()

In [ ]:
conn.close()

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv('out.csv', sep=',')

,goods_genre_id,avg_price
0,12,1.381663e+06
1,15,1.197012e+06
2,16,1.315036e+06
3,19,2.957700e+04
4,20,1.541658e+05
5,21,2.481760e+05
6,22,5.210007e+04
7,23,1.674405e+05
8,24,1.646757e+05
9,26,7.189457e+04
